# It's my _____!!!!!

```
ZFun{tomori_capture_sakiko!!!}
base64: WkZ1bnt0b21vcmlfY2FwdHVyZV9zYWtpa28hISF9
```

摸鱼随便搓了一个图片隐写（~~主要还是玩梗~~

flag经过base64编码之后，随手找个在线工具生成二维码

以下是出题过程的简单分析

In [1]:
import cv2
import numpy as np
import random

carrier = cv2.imread("carrier.jpg") # 表情包本体
payload = cv2.imread("payload.png") # 包含flag的二维码
payload = cv2.cvtColor(payload, cv2.COLOR_BGR2GRAY) # 因为在线工具生成的是BGR格式的图像，所以转成灰度图，方便后续处理
payload = cv2.resize(payload, (carrier.shape[1] // 4, carrier.shape[0] // 4)) # 原图太大了，resize一下

In [17]:
offset_x, offset_y = 114, 114 # 特意不放在左上角，防止某些工具一眼看出lsb
for x in range(0, payload.shape[0]):
    for y in range(0, payload.shape[1]):
        for c in range(0, 3): # 对BGR三个通道
            carrier[offset_x + x][offset_y + y][c] = carrier[offset_x + x][offset_y + y][c] & 254 # 先把最低位清零
        if payload[x][y] > 127: # 二维码是黑白图，所以大于127的认为是1
            c = random.randrange(0, 3) # 随机选一个通道
            carrier[offset_x + x][offset_y + y][c] = carrier[offset_x + x][offset_y + y][c] | 1 # 如果二维码的该像素为1，则把输出的该像素最低位置1
cv2.imwrite("result.png", carrier) # 输出结果，即为题目附件

True

那么，在拿到附件之后从什么地方下手呢？

~~其实图片隐写的方法有很多，但是在CTF中常见的也就几种（~~

别的工具一把梭不出来，那就试试LSB，因为有三个通道，所以分别对三个通道提取LSB

In [18]:
result = cv2.imread("result.png")
for c in range(0,3):
  qrcode = np.zeros((result.shape[0], result.shape[1]), np.uint8) # 创建一个空白图像
  for x in range(0, result.shape[0]):
      for y in range(0, result.shape[1]):
            if result[x][y][c]&1 == 1: # 如果最低位为1
              qrcode[x][y] = 255 # 则该像素为白色
  cv2.imwrite(f"qrcode_{c}.png", qrcode)

然后发现在每个输出的文件中，都可以看到一个二维码的定位标记，所以可以推断此处肯定有二维码，尝试把所有白色像素叠加

In [19]:
result = cv2.imread("result.png")
qrcode = np.zeros((result.shape[0], result.shape[1]), np.uint8)
for x in range(0, result.shape[0]):
    for y in range(0, result.shape[1]):
        if any(i&1 == 1 for i in result[x][y]):
            qrcode[x][y] = 255
cv2.imwrite("qrcode.png", qrcode)

True

得到了一个二维码，扫描之后，通过base64解码得到flag